In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np


In [2]:
#read data from conpressed file
vehicle_data = pd.read_csv("vehicle_data.csv.gz",
               compression="gzip")

#strip extra spaces in field
vehicle_data['fuel_type'].str.rstrip().astype(str)
vehicle_data['elec_hybrid']= vehicle_data['fuel_type'].isin(['ELECTRIC','NONE','OTHER'])
vehicle_data


,vin,registration_class,zip,model_year,body_type,fuel_type,elec_hybrid
0,999407G3573,PAS,12866,1976,SUBN,GAS,False
1,9992313,PAS,13316,1924,CONV,GAS,False
2,998867,PAS,11501,1952,2DSD,GAS,False
3,9983692,PAS,10940,1936,2DSD,GAS,False
4,99782010525,PAS,11225,1978,2DSD,GAS,False
...,...,...,...,...,...,...,...
9375975,AAAAAAKE201612240,PAS,14621,1973,2DSD,GAS,False
9375976,,PAS,14120,1941,4DSD,GAS,False
9375977,,SRF,10580,1967,2DSD,GAS,False
9375978,,PAS,14047,1958,CONV,GAS,False


In [10]:
#Seperate Data by fuel_clas
elec_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == True]
std_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == False]

#Rename Columns
elec_df = elec_df.rename(columns={"elec_hybrid": "Green"})
elec_df['Green'] =  1
elec_df['Green'].replace(np.nan,0)

std_df = std_df.rename(columns={"elec_hybrid": "Standard"})
std_df['Standard'] = 1
std_df['Standard'].replace(np.nan,0)

#group by
new_elec_df = elec_df.groupby('zip').agg({'Green':'size'})
new_std_df = std_df.groupby('zip').agg({'Standard':'size'})

#Merge the 2 data sets
combined_df = pd.merge(new_elec_df, new_std_df, on=['zip'], how="left" )
combined_df.replace(np.nan,0)


,Green,Standard
zip,,
1035,1,0.0
2534,1,0.0
4107,1,0.0
6390,2,429.0
6437,1,3.0
...,...,...
90069,1,11.0
90211,1,3.0
90265,1,5.0


In [59]:
#Add AQI data
aqi_df = pd.read_csv("AQIndex2020.csv", index_col=1)
aqi_df.replace(np.nan,0)

#merge with above
type_aqi_df = pd.merge(aqi_df, combined_df, on=['zip'], how = "left" )
type_aqi_df = type_aqi_df.drop(columns =['Unnamed: 0','Category Number', 'Category Name','Pollutant'])
type_aqi_df


,AQI,Green,Standard
zip,,,
12023,46,4.0,1510.0
12024,46,1.0,128.0
12025,46,4.0,4063.0
12027,46,15.0,3096.0
12028,46,NaN,NaN
...,...,...,...
13165,45,1.0,6955.0
13166,45,2.0,3852.0
13318,45,NaN,NaN


In [49]:
fuel_aqi_df = type_aqi_df

fuel_aqi_df['Percent_Green']= (fuel_aqi_df['Green']/fuel_aqi_df['Standard'])

#clean Up formatting drop NaN rows
fuel_aqi_df['Percent_Green']= fuel_aqi_df['Percent_Green'].astype(float).astype(float).map("{:.2%}".format)
dropped_df = fuel_aqi_df.dropna()
dropped_df

,AQI,Green,Standard,Percent_Green
zip,,,,
12023,46,4.0,1510.0,0.26%
12024,46,1.0,128.0,0.78%
12025,46,4.0,4063.0,0.10%
12027,46,15.0,3096.0,0.48%
12029,46,4.0,864.0,0.46%
...,...,...,...,...
13159,45,5.0,3754.0,0.13%
13160,45,1.0,1419.0,0.07%
13165,45,1.0,6955.0,0.01%


In [54]:
#zip code with largest concentration of green Vehicles
dropped_df['Percent_Green'].max()

'1.62%'